In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
from datetime import datetime, timedelta
import requests
import pandas as pd

# read bearer token for authentication
#with open('bearer_token.txt') as fp:
 #   BEARER_TOKEN = fp.read()
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAALqAYQEAAAAAJcPBWd6awcepf6nYpKm2sHSQYfw%3DilKIYlnOALlhxSSSS3kEsB8tP1NVBUySmWpL9yyLspALZfxyKb'
    
# setup the API request
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query': '(#tsla) (lang:en)',
    'max_results': '100',
    'tweet.fields': 'created_at,lang'
}

dtformat = '%Y-%m-%dT%H:%M:%SZ'  # the date format string required by twitter

# we use this function to subtract 60 mins from our datetime string
def time_travel(now, mins):
    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)
    
now = datetime.now()  # get the current datetime, this is our starting point
last_week = now - timedelta(days=7)  # datetime one week ago = the finish line
now = now.strftime(dtformat)  # convert now datetime to format for API

df = pd.DataFrame()  # initialize dataframe to store tweets

def get_data(tweet):
    data = {'tweet_id': tweet['id'],
            'created_at': tweet['created_at'],
            'text': tweet['text']
            #,'author_id' : tweet['author_id']
           }
    return data

i=0
while i<2:
    i+=1
    if datetime.strptime(now, dtformat) < last_week:
        # if we have reached 7 days ago, break the loop
        break
    pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
    # assign from and to datetime parameters for the API
    params['start_time'] = pre60 
    params['end_time'] = now
    #print(pre60, now)
    response = requests.get(endpoint,
                            params=params,
                            headers=headers)  # send the request
    now = pre60  # move the window 60 minutes earlier
    # iteratively append our tweet data to our dataframe
    for tweet in response.json()['data']:
        row = get_data(tweet)  # we defined this function earlier
        df = df.append(row, ignore_index=True)
    
#print(response.json())

2022-03-05T12:41:23Z 2022-03-05T13:41:23Z
2022-03-05T11:41:23Z 2022-03-05T12:41:23Z


In [3]:
df.head()

,tweet_id,created_at,text
0,1500103445952483332,2022-03-05T13:38:26.000Z,@foso_defensivo $TELL &gt;&gt;&gt;&gt;&gt; #tsla
1,1500101377955090434,2022-03-05T13:30:13.000Z,WallStreetBets - Daily Breakout Mentions (3/4/...
2,1500098272215588866,2022-03-05T13:17:53.000Z,RT @GailAlfarATX: Elon Musk explains autonomou...
3,1500087200528187393,2022-03-05T12:33:53.000Z,@SawyerMerritt I wish Rivian good luck. No one...
4,1500083760661884928,2022-03-05T12:20:13.000Z,$TSLA broke March 2020 uptrend last week and b...


In [4]:
df['text'][0]

'@foso_defensivo $TELL &gt;&gt;&gt;&gt;&gt; #tsla'